In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb

# 1. CSV 불러오기
df = pd.read_csv("C:/Users/NM333-67/Desktop/personalized-tour-ai/All_reviews_04_trust_score.csv")

# 2. 신뢰 리뷰만 필터링
trusted_df = df[df['신뢰라벨'] == 1].dropna(subset=['리뷰내용']).reset_index(drop=True)
texts = trusted_df['리뷰내용'].tolist()

# 3. E5-multilingual 모델 로드
model = SentenceTransformer("intfloat/multilingual-e5-base")

# 4. E5는 prefix 필수 ("passage: ")
prefixed_texts = [f"passage: {text}" for text in texts]

# 5. 임베딩 생성
embeddings = model.encode(prefixed_texts, convert_to_tensor=False)

# 6. ChromaDB Persistent 클라이언트 및 컬렉션 생성
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="C:/Users/NM333-67/Desktop/personalized-tour-ai/05_vector_db/chroma_storage")

if "trusted_reviews" in [c.name for c in chroma_client.list_collections()]:
    chroma_client.delete_collection(name="trusted_reviews")

collection = chroma_client.create_collection(name="trusted_reviews")

# 7. 확장 메타데이터 컬럼 지정
metadata_columns = [
    '가게명', '별점', '작성자',
    '사용자총리뷰수', '총평점', '주소',
    '리뷰 페이지', 'trust_score'
]

# 8. ChromaDB에 저장
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    metadata = trusted_df.iloc[i][metadata_columns].to_dict()
    collection.add(
        embeddings=[embedding],
        documents=[text],
        metadatas=[metadata],
        ids=[str(i)]
    )

print(f"총 {len(texts)}개의 신뢰 리뷰가 벡터 DB에 저장되었습니다.")
print("저장된 벡터 수:", collection.count())


c:\Users\NM333-67\anaconda3\envs\personalized-tour-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\NM333-67\AppData\Local\Temp\ipykernel_16732\489828497.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/NM333-67/Desktop/personalized-tour-ai/All_reviews_04_trust_score.csv")


총 17757개의 신뢰 리뷰가 벡터 DB에 저장되었습니다.
저장된 벡터 수: 17757


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# 1. ChromaDB 디스크 클라이언트 연결
chroma_client = chromadb.PersistentClient(path="C:/Users/NM333-67/Desktop/personalized-tour-ai/05_vector_db/chroma_storage")
collection = chroma_client.get_collection(name="trusted_reviews")

# 2. 컬렉션 정보 확인 (선택적)
print("컬렉션 목록:", chroma_client.list_collections())
print("저장된 벡터 수:", collection.count())

# 3. 임베딩 모델 로드 (저장할 때 썼던 동일 모델이어야 함!)
model = SentenceTransformer("intfloat/multilingual-e5-base")

# 4. 사용자 질문 입력
query = "혼밥하기 좋은 식당 알려줘"

# 5. E5 계열은 query prefix 필요
query_embedding = model.encode(f"query: {query}", convert_to_tensor=False)

# 6. top 5 유사 리뷰 검색
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

# 7. 결과 출력
for doc, meta in zip(results['documents'][0], results['metadatas'][0]):
    print(f"가게명: {meta['가게명']}")
    print(f"별점: {meta['별점']} / 작성자: {meta['작성자']}")
    print(f"주소: {meta['주소']}")
    print(f"리뷰 페이지: {meta['리뷰 페이지']}")
    print(f"리뷰 내용: {doc}")
    print(f"trust_score: {meta['trust_score']}")
    print("──" * 30)


c:\Users\NM333-67\anaconda3\envs\personalized-tour-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


컬렉션 목록: [Collection(name=trusted_reviews)]
저장된 벡터 수: 17757
가게명: 어보
별점: 5.0 / 📝 작성자: ^_^
주소: 부산 기장군 일광읍 일광로 582-22
리뷰 페이지: https://place.map.kakao.com/2004010607#comment
리뷰 내용: 부모님 모시고 가기 좋은 식당 같아용
trust_score: 0.686
────────────────────────────────────────────────────────────
가게명: 삼원칼국수
별점: 5.0 / 📝 작성자: IllllIl
주소: 부산 부산진구 가야대로765번길 27 1층
리뷰 페이지: https://place.map.kakao.com/735226808#comment
리뷰 내용: 혼밥하기 좋은 곳 김치 매운데 맛있음..굿
trust_score: 0.75
────────────────────────────────────────────────────────────
가게명: 우정
별점: 5.0 / 📝 작성자: ㅎㅇㅎㅇ
주소: 부산 해운대구 센텀동로 9
리뷰 페이지: https://place.map.kakao.com/20542979#comment
리뷰 내용: 맛있고 양 많고 혼밥하기 좋아요
trust_score: 0.75
────────────────────────────────────────────────────────────
가게명: 각방
별점: 5.0 / 📝 작성자: 호우호우
주소: 부산 부산진구 동천로107번길 12-5 1층
리뷰 페이지: https://place.map.kakao.com/357123994#comment
리뷰 내용: 일단 먼저 혼밥 하시려면 여기 완전 강추함 혼밥할 수 있는 테이블 있고 가게 그렇게 크지않아서 아주 좋음 주문도 테이블에서 주문+카드결제까지 삽가능함ㅇㅇ 연어덮밥이랑 닭꼬치덮밥 시켰는데 연어 상태 생각보다 좋음 좋아요  닭꼬치는 내가 맵고짜고 단거 좋아해서 그른가 난 넘무 맛도리여쑴…싱거운거 좋아하면 

### 새 데이터 추가 코드

In [ ]:
# 디스크 DB 연결
chroma_client = chromadb.PersistentClient(path="C:/Users/NM333-67/Desktop/personalized-tour-ai/05_vector_db/chroma_storage")
collection = chroma_client.get_collection(name="trusted_reviews")

# 새 데이터 예시
new_text = "passage: 새로 등록된 맛집 리뷰 예시입니다."
new_embedding = model.encode(new_text, convert_to_tensor=False)
new_metadata = {
    "가게명": "신규맛집",
    "별점": 4.7,
    "작성자": "새로운 사용자",
    "사용자총리뷰수": 10,
    "총평점": 4.8,
    "주소": "서울시 예시구",
    "리뷰 페이지": "https://example.com",
    "trust_score": 0.95
}

# 추가
collection.add(
    embeddings=[new_embedding],
    documents=[new_text],
    metadatas=[new_metadata],
    ids=["new_001"]
)

print("새 데이터가 디스크 DB에 추가되었습니다.")
